# Technical indicator classes
In order to use lots of indicators in one strategy, we have to make a class that handles calculations for each technical indicator. That will make the code simpler.

In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#plt.style.use("seaborn")
#import sys

In [76]:
#load raw data 
def load_data(): 
    start = "2022-01-31"
    end = "2023-06-30"
    raw = pd.read_csv("5m.csv", parse_dates = ["Date"], index_col = "Date").dropna()
    raw = raw.loc[start:end]
    raw = raw[["Close"]].rename(columns={"Close": "price"}) #just use these columns
    raw["returns"] = np.log(raw.price / raw.price.shift(1))
    raw

In [77]:
#Simple Moving Average
class SMA():
    
    def __init__(self, data, SMA_S, SMA_L, column):
        self.data = data # Dataframe
        self.SMA_S = SMA_S # short SMA
        self.SMA_L = SMA_L # long SMA
        self.column = column # column to use SMA
        self.data["SMA_S"] = np.nan
        self.data["SMA_L"] = np.nan
        
    def calculate(self): #calculate for all dataframe
        self.data["SMA_S"] = self.data[self.column].rolling(self.SMA_S).mean()
        self.data["SMA_L"] = self.data[self.column].rolling(self.SMA_L).mean()
        #DONT DROP NA BECAUSE OTHER INDICATORS NEED THAT ROWS!!!
    
    def calculate_for_last_row(self): #calculate just for last row
        self.data["SMA_S"].iloc[-self.SMA_S:] = self.data[self.column].iloc[-self.SMA_S:].rolling(self.SMA_S).mean()
        self.data["SMA_L"].iloc[-self.SMA_L:] = self.data[self.column].iloc[-self.SMA_L:].rolling(self.SMA_L).mean()
sma = SMA(
        data = raw,
        SMA_S = 50,
        SMA_L = 200,
        column = "price"
         )
load_data() #resets data
#sma.calculate() #calculate for all dataframe
sma.calculate_for_last_row() #calculate just for last row
raw #notice that original dataframe is changed

,price,returns,SMA_S,SMA_L
Date,,,,
2022-07-12 11:15:00,19729.8,NaN,NaN,NaN
2022-07-12 11:20:00,19713.8,-0.000811,NaN,NaN
2022-07-12 11:25:00,19772.7,0.002983,NaN,NaN
2022-07-12 11:30:00,19823.3,0.002556,NaN,NaN
2022-07-12 11:35:00,19766.9,-0.002849,NaN,NaN
...,...,...,...,...
2022-08-16 04:10:00,24550.0,0.002039,NaN,NaN
2022-08-16 04:15:00,24450.0,-0.004082,NaN,NaN
2022-08-16 04:20:00,24508.3,0.002382,NaN,NaN


In [84]:
#Exponentially Weighted Moving Average
class EWMA():
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html
    def __init__(self, data, EWMA_S, EWMA_L, alpha_s, alpha_l, column):
        self.data = data # Dataframe
        self.EWMA_S = EWMA_S #short EWMA
        self.EWMA_L = EWMA_L #long EWMA
        self.alpha_s = alpha_s #alpha for short EWMA
        self.alpha_l = alpha_l #alpha for long EWMA
        self.column = column # column to use SMA
        self.data["EWMA_S"] = np.nan
        self.data["EWMA_L"] = np.nan
        
    def calculate(self): #calculate for all dataframe
        self.data["EWMA_S"] = self.data[self.column].ewm(alpha = self.alpha_s).mean()
        self.data["EWMA_L"] = self.data[self.column].ewm(alpha = self.alpha_l).mean()
        #DONT DROP NA BECAUSE OTHER INDICATORS NEED THAT ROWS!!!
    
    def calculate_for_last_row(self): #calculate just for last row
        self.data["EWMA_S"].iloc[-1:] = self.data[self.column].iloc[-1:].ewm(alpha = self.alpha_s).mean()
        self.data["EWMA_L"].iloc[-1:] = self.data[self.column].iloc[-1:].ewm(alpha = self.alpha_l).mean()
        
ewma = EWMA(
        data = raw,
        EWMA_S = 7,
        EWMA_L = 25,
        alpha_s = 2/3,
        alpha_l = 2/3,
        column = "price"
         )
load_data() #resets data
ewma.calculate() #calculate for all dataframe
#ewma.calculate_for_last_row() #calculate just for last row
raw #notice that original dataframe is changed

,price,returns,SMA_S,SMA_L,EWMA_S,EWMA_L
Date,,,,,,
2022-07-12 11:15:00,19729.8,NaN,NaN,NaN,19729.800000,19729.800000
2022-07-12 11:20:00,19713.8,-0.000811,NaN,NaN,19717.800000,19717.800000
2022-07-12 11:25:00,19772.7,0.002983,NaN,NaN,19755.807692,19755.807692
2022-07-12 11:30:00,19823.3,0.002556,NaN,NaN,19801.365000,19801.365000
2022-07-12 11:35:00,19766.9,-0.002849,NaN,NaN,19778.293388,19778.293388
...,...,...,...,...,...,...
2022-08-16 04:10:00,24550.0,0.002039,NaN,NaN,24530.030906,24530.030906
2022-08-16 04:15:00,24450.0,-0.004082,NaN,NaN,24476.676969,24476.676969
2022-08-16 04:20:00,24508.3,0.002382,NaN,NaN,24497.758990,24497.758990
